# setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

sklearn version:  0.24.2
TF version:  2.8.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


# Get the data

In [2]:
#작성자 전민재, hsh
import csv
def load_Emist(exsitNumpy=False, needTranspose=True):
    if (exsitNumpy == False):
        #"./emnist-byclass-test.csv"
        #train-set
        #"공용/datasets/Emnist/" 위치에 csv 저장 
        csv_train_data_file = open("../공용/datasets/Emnist/train_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        csv_train_label_file = open("../공용/datasets/Emnist/train_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
    
        
        f_train = csv.reader(csv_train_data_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        l_train=csv.reader(csv_train_label_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        #valid-set
    
        csv_valid_data_file = open("../공용/datasets/Emnist/valid_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        csv_valid_label_file = open("../공용/datasets/Emnist/valid_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        
        f_valid = csv.reader(csv_valid_data_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        l_valid = csv.reader(csv_valid_label_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        X_train=[]
        y_train=[]
        X_valid=[]
        y_valid=[]
        for i, row in enumerate(f_train):
            #행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx]=int(char)
            #train
            #data 추가    
            X_train.append(row)
            
        for i, row in enumerate(f_valid):
            #행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx]=int(char)
            #valid
            #data 추가    
            X_valid.append(row)


        for i, row in enumerate(l_train):

            #train
            #label 추가
            y_train.append(int(row[0]))
            
        for i, row in enumerate(l_valid):

            #valid
            #label 추가
            y_valid.append(int(row[0]))
            
            
        

        #train
        X_train=np.array(X_train,dtype=np.uint8)
        X_train=X_train.reshape(-1,28,28)
        #valid
        X_valid=np.array(X_valid,dtype=np.uint8)
        X_valid=X_valid.reshape(-1,28,28)
        
        csv_train_data_file.close()
        csv_train_label_file.close()
        csv_valid_data_file.close()
        csv_valid_label_file.close()
        
        #kaggle dataset이 시계반대방향으로 90도 회전 되있고 상하 반전 되어있음
        def rotate_90(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[c][N-1-r] = m[r][c]
            return np.array(ret,dtype=np.uint8)

        def vreflect(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[r][c] = m[r][N-1-c]
            return np.array(ret,dtype=np.uint8)
        
        if needTranspose == True:
            #train
            #회전
            for idx,i in enumerate(X_train):
                X_train[idx]=rotate_90(i)
            #상하반전
            for idx,i in enumerate(X_train):
                X_train[idx]=vreflect(i)

            #valid
            #회전
            for idx,i in enumerate(X_valid):
                X_valid[idx]=rotate_90(i)
            #상하반전
            for idx,i in enumerate(X_valid):
                X_valid[idx]=vreflect(i)

       
        
        
        #저장용
        #train
        #(533917,28,28)
        
        np.save('../공용/X_train',np.array(X_train,dtype=np.uint8))
        np.save('../공용/y_train',np.array(y_train,dtype=np.uint8))
        #valid
        np.save('../공용/X_valid',np.array(X_valid,dtype=np.uint8))
        np.save('../공용/y_valid',np.array(y_valid,dtype=np.uint8))
        
   
    X_train=np.load('./X_train.npy')
    y_train=np.load('./y_train.npy')
    X_valid=np.load('./X_valid.npy')
    y_valid=np.load('./y_valid.npy')
    
    return X_train, y_train, X_valid, y_valid


In [3]:
#황성현
#npy 파일이 존재할 경우 exsitNumpy = True, 업으면 False
X_train, y_train, X_valid, y_valid = load_Emist(exsitNumpy=True)
X_train.shape,X_train.dtype

((533917, 28, 28), dtype('uint8'))

In [4]:
#황성현
X_train=tf.reshape(X_train,[-1,28,28,1])
X_valid=tf.reshape(X_valid,[-1,28,28,1])

In [5]:
#황성현,jmj
X_train.shape,X_valid.shape,X_train.dtype

(TensorShape([533917, 28, 28, 1]), TensorShape([164015, 28, 28, 1]), tf.uint8)

In [6]:
#황성현, jmj
X_train_resize=np.uint8(tf.image.resize(X_train, [32, 32]))
X_valid_resize=np.uint8(tf.image.resize(X_valid, [32, 32]))


In [7]:
#황성현
X_train_resize.shape,X_valid_resize.dtype,X_valid_resize.shape

((533917, 32, 32, 1), dtype('uint8'), (164015, 32, 32, 1))

# train the model

# fine tuning

# SMV2L Batch64

In [8]:
#황성현
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            
            ]
        self.strides_layers=[
           ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n
        
    def build(self,batch_input_shape):
        print(self.name,": batch_input_shape =",batch_input_shape)
        
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1, 
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=1,padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c,kernel_size=1, strides=1,padding="SAME", activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=self.s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c, kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [ ]:
#황성현
#multygpu
strategy = tf.distribute.MirroredStrategy()


In [20]:
with strategy.scope():

    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_10 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [21]:
#황성현 
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_BATCH64_%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
batch_size=64
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam_batch64_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam_batch64_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history=smv2_l.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),                   
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb])


logs/SMV2L_BATCH64_20220602-150219
Epoch 1/300
8343/8343 [==============================] - 599s 71ms/step - loss: 0.7806 - accuracy: 0.7545 - val_loss: 0.5223 - val_accuracy: 0.8198
Epoch 2/300
8343/8343 [==============================] - 590s 71ms/step - loss: 0.4600 - accuracy: 0.8364 - val_loss: 0.4372 - val_accuracy: 0.8459
Epoch 3/300
8343/8343 [==============================] - 592s 71ms/step - loss: 0.4219 - accuracy: 0.8479 - val_loss: 0.4013 - val_accuracy: 0.8527
Epoch 4/300
8343/8343 [==============================] - 591s 71ms/step - loss: 0.4005 - accuracy: 0.8537 - val_loss: 0.3908 - val_accuracy: 0.8574
Epoch 5/300
8343/8343 [==============================] - 591s 71ms/step - loss: 0.3867 - accuracy: 0.8576 - val_loss: 0.3833 - val_accuracy: 0.8604
Epoch 6/300
8343/8343 [==============================] - 594s 71ms/step - loss: 0.3770 - accuracy: 0.8603 - val_loss: 0.3822 - val_accuracy: 0.8600
Epoch 7/300
8343/8343 [==============================] - 592s 71ms/step - los

# SMV2L Batch128

In [22]:
#황성현
with strategy.scope():

    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_12 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [23]:
#황성현 
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_BATCH128_%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
batch_size=128
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam_batch128_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam_batch128_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history=smv2_l.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),                   
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb])


logs/SMV2L_BATCH128_20220602-221702
Epoch 1/300
4172/4172 [==============================] - 311s 73ms/step - loss: 0.8230 - accuracy: 0.7429 - val_loss: 0.6796 - val_accuracy: 0.7828
Epoch 2/300
4172/4172 [==============================] - 298s 72ms/step - loss: 0.4592 - accuracy: 0.8369 - val_loss: 0.4403 - val_accuracy: 0.8426
Epoch 3/300
4172/4172 [==============================] - 302s 72ms/step - loss: 0.4183 - accuracy: 0.8478 - val_loss: 0.4197 - val_accuracy: 0.8469
Epoch 4/300
4172/4172 [==============================] - 299s 72ms/step - loss: 0.3952 - accuracy: 0.8551 - val_loss: 0.4032 - val_accuracy: 0.8524
Epoch 5/300
4172/4172 [==============================] - 301s 72ms/step - loss: 0.3806 - accuracy: 0.8588 - val_loss: 0.3786 - val_accuracy: 0.8609
Epoch 6/300
4172/4172 [==============================] - 297s 71ms/step - loss: 0.3682 - accuracy: 0.8627 - val_loss: 0.3916 - val_accuracy: 0.8580
Epoch 7/300
4172/4172 [==============================] - 306s 73ms/step - lo

# SMV2P Batch64

In [24]:
#황성현
with strategy.scope():
    n_classes=62
    smv2_p=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_p.summary()
    smv2_p.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 32)
IRB4 : batch_input_shape = (None, 4, 4, 64)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_14 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 32)   

In [25]:
#황성현 
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2P_BATCH64_%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
batch_size=64
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam_batch64_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam_batch64_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history=smv2_p.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),                   
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb])

logs/SMV2P_BATCH64_20220603-002710
Epoch 1/300
8343/8343 [==============================] - 616s 73ms/step - loss: 0.7312 - accuracy: 0.7669 - val_loss: 0.5126 - val_accuracy: 0.8257
Epoch 2/300
8343/8343 [==============================] - 605s 73ms/step - loss: 0.4528 - accuracy: 0.8395 - val_loss: 0.4240 - val_accuracy: 0.8500
Epoch 3/300
8343/8343 [==============================] - 603s 72ms/step - loss: 0.4164 - accuracy: 0.8498 - val_loss: 0.4293 - val_accuracy: 0.8393
Epoch 4/300
8343/8343 [==============================] - 609s 73ms/step - loss: 0.3964 - accuracy: 0.8554 - val_loss: 0.3867 - val_accuracy: 0.8574
Epoch 5/300
8343/8343 [==============================] - 607s 73ms/step - loss: 0.3848 - accuracy: 0.8585 - val_loss: 0.3867 - val_accuracy: 0.8581
Epoch 6/300
8343/8343 [==============================] - 607s 73ms/step - loss: 0.3750 - accuracy: 0.8610 - val_loss: 0.3836 - val_accuracy: 0.8607
Epoch 7/300
8343/8343 [==============================] - 607s 73ms/step - los

# SMV2P Batch128

In [26]:
#황성현
with strategy.scope():
    n_classes=62
    smv2_p=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_p.summary()
    smv2_p.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 32)
IRB4 : batch_input_shape = (None, 4, 4, 64)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_16 (Bat  (None, 32, 32, 1)        4         
 chNormalization)                                                
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 32)   

In [27]:
#황성현 
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2P_BATCH128_%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
batch_size=128
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam_batch128_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam_batch128_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history=smv2_p.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),                   
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb])

logs/SMV2P_BATCH128_20220603-051048
Epoch 1/300
4172/4172 [==============================] - 303s 71ms/step - loss: 0.7617 - accuracy: 0.7585 - val_loss: 0.5404 - val_accuracy: 0.8100
Epoch 2/300
4172/4172 [==============================] - 290s 70ms/step - loss: 0.4468 - accuracy: 0.8402 - val_loss: 0.4251 - val_accuracy: 0.8492
Epoch 3/300
4172/4172 [==============================] - 296s 71ms/step - loss: 0.4120 - accuracy: 0.8503 - val_loss: 0.4080 - val_accuracy: 0.8503
Epoch 4/300
4172/4172 [==============================] - 292s 70ms/step - loss: 0.3918 - accuracy: 0.8564 - val_loss: 0.3900 - val_accuracy: 0.8554
Epoch 5/300
4172/4172 [==============================] - 293s 70ms/step - loss: 0.3764 - accuracy: 0.8605 - val_loss: 0.3790 - val_accuracy: 0.8608
Epoch 6/300
4172/4172 [==============================] - 295s 71ms/step - loss: 0.3651 - accuracy: 0.8638 - val_loss: 0.3891 - val_accuracy: 0.8572
Epoch 7/300
4172/4172 [==============================] - 293s 70ms/step - lo